In [9]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [53]:
def plot_cer_co2(R, number):
    plt.figure(1,figsize=(16, 14))
    ax1 = plt.subplot(212)
    plt.scatter(R['Time (min)']/60, R['Bioreactor ' + str(number) + ' - CER'])
    plt.setp(ax1.get_xticklabels(), fontsize=6)
    plt.xlabel('Time (hour)', fontsize = 20)
    plt.ylabel('CER (CO2 % / min)', fontsize = 20)
    plt.tick_params(labelsize=15)
    plt.xticks(np.arange(0, len(R), step=5))

    ax2 = plt.subplot(211, sharex=ax1)
    plt.scatter(R['Time (min)']/60, R['Bioreactor ' + str(number) + ' - Off-gas CO2%'])
    plt.setp(ax2.get_xticklabels(), visible=False)
    plt.ylabel('CO2 %', fontsize = 20)
    plt.title('Reactor ' + str(number), fontsize = 20)
    plt.tick_params(labelsize=15)

    plt.show()

In [54]:
def calculate_mu(R, number):
    data_frame_selected_values = R[np.isfinite(R['Bioreactor ' + str(number) + ' - CER'])]
    data_frame_selected_values.reset_index(inplace=True, drop=True)
    data_frame_selected_values = data_frame_selected_values.copy()
    data_frame_selected_values['Time (min)'] = data_frame_selected_values['Time (min)']-data_frame_selected_values['Time (min)'][0]

    tCER = []
    tCER.append(0)  # Here set the initial value of tCER if we have that.

    for i in range(0, (len(data_frame_selected_values['Time (min)']) - 1)):

        tCER_i = ((data_frame_selected_values['Bioreactor ' + str(number) + ' - CER'][i] + data_frame_selected_values['Bioreactor ' + str(number) + ' - CER'][i + 1]) / 2) * (data_frame_selected_values['Time (min)'][i + 1] - data_frame_selected_values['Time (min)'][i]) + tCER[i]
        tCER.append(tCER_i)

    mu = data_frame_selected_values['Bioreactor ' + str(number) + ' - CER'] / tCER  # [1/min]
    
    return data_frame_selected_values, mu

In [55]:
def plot_mu(data_frame_selected_values, mu):
    
    plt.figure(1,figsize=(16, 7))
    plt.scatter(data_frame_selected_values['Time (min)']/60, mu*60)
    #plt.setp(ax2.get_xticklabels(), visible=False)
    plt.ylabel('mu (1/hour)', fontsize = 20)
    plt.xlabel('Time (hour)', fontsize = 20)
    #plt.title('Reactor ' + str(number), fontsize = 20)
    plt.tick_params(labelsize=15)
    plt.xticks(np.arange(0, 120, step=5))


    plt.show()
    

In [ ]:
def plot_mu_smaller_interval(data_frame_selected_values, mu):

    plt.figure(1,figsize=(16, 7))
    plt.scatter(data_frame_selected_values['Time (min)'][0:40]/60, mu[0:40]*60)
    #plt.setp(ax2.get_xticklabels(), visible=False)
    plt.ylabel('mu (1/hour)', fontsize = 20)
    plt.xlabel('Time (hour)', fontsize = 20)
    #plt.title('Reactor ' + str(number), fontsize = 20)
    plt.tick_params(labelsize=15)


    plt.show()

In [ ]:
def plot_mu_smaller_interval_R9_to_R12(data_frame_selected_values, mu):

    plt.figure(1,figsize=(16, 7))
    plt.scatter(data_frame_selected_values['Time (min)'][0:60]/60, mu[0:60]*60)
    #plt.setp(ax2.get_xticklabels(), visible=False)
    plt.ylabel('mu (1/hour)', fontsize = 20)
    plt.xlabel('Time (hour)', fontsize = 20)
    #plt.title('Reactor ' + str(number), fontsize = 20)
    plt.tick_params(labelsize=15)


    plt.show()

In [ ]:
def time_converter(sample_timepoints):
    sample_timepoints = pd.ExcelFile(sample_timepoints)
    sample_timepoints = sample_timepoints.parse('Sheet1')


    # alle starter fra 14:30, derfor kan denne metode bruges til alle 
    # Create a dataframe to put the result in (which is the converted time to decimals)
    sample_timepoints_decimals = pd.DataFrame(columns=['time (hours)'])


    # This loops through all datapoints,
    # convert it string, splits and calculate the time to minutes in decimals
    for i in range(len(sample_timepoints)):
        t = sample_timepoints['timepoints'][i]
        t = str(t)
        (h, m, s) = t.split(':')
        result = int(h) * 60 + int(m) + int(s) / 60
        sample_timepoints_decimals.loc[
        i, ['time (hours)']] = result 

    # To get hours instead and add 24 hours to next day
    time_hours = sample_timepoints_decimals/60
    next_day = time_hours[8:11] + 24
    time_hours = time_hours.drop(time_hours.tail(3).index,inplace=False)

    # Because our sample starts at 11 something, we substract 11 and then add the 44.5 
    # because CO2 data frame starts from 43 hours, and sample timestarts from 11 something
    time_hours = time_hours.append(next_day, ignore_index=True)
    time_hours = time_hours-11+44.5
    return time_hours

In [3]:
def plot_biomass(sample_timepoints_R, biomass_R):

    plt.figure(1,figsize=(16, 7))
    plt.scatter(sample_timepoints_R,biomass_R)
    plt.ylabel('Biomass (g DW/L)', fontsize = 20)
    plt.xlabel('Time (hour)', fontsize = 20)
    
    plt.show()


In [ ]:
def time_converter_R9_to_R12(sample_timepoints):
    sample_timepoints = pd.ExcelFile(sample_timepoints)
    sample_timepoints = sample_timepoints.parse('Sheet1')

    #print(sample_timepoints)
    # Because all data sets starts from 14:30, the method can be used for all data sets
    # Create a dataframe to put the result in (which is the converted time to decimals)
    sample_timepoints_decimals = pd.DataFrame(columns=['time (hours)'])

    # This loops through all datapoints,
    # convert it string, splits and calculate the time to minutes in decimals
    for i in range(len(sample_timepoints)):
        t = sample_timepoints['timepoints'][i]
        t = str(t)
        (h, m, s) = t.split(':')
        result = int(h) * 60 + int(m) + int(s) / 60
        sample_timepoints_decimals.loc[
        i, ['time (hours)']] = result 

    # To get hours instead and add 24 hours to get to the next day
    time_hours = sample_timepoints_decimals/60
    next_day = time_hours[7:18] + 24
    time_hours = time_hours.drop(time_hours.tail(11).index,inplace=False)
    time_hours = time_hours.append(next_day, ignore_index=True)

    # And add 24 to the 3 day
    next_day = time_hours[17:18] + 24
    time_hours = time_hours.drop(time_hours.tail(1).index,inplace=False)
    time_hours = time_hours.append(next_day, ignore_index=True)
    time_hours = time_hours-11+44.5
    return time_hours

In [4]:
def semilog_plot_biomass(sample_timepoints_R, biomass_R):
    # Take the logarithmic of biomass, np.log is the same as ln 
    plt.figure(1,figsize=(16, 7))
    plt.scatter(sample_timepoints_R,np.log(biomass_R))
    plt.ylabel('Biomass (g DW/L)', fontsize = 20)
    plt.xlabel('Time (hour)', fontsize = 20)
    
    plt.show()

In [13]:
def lin_reg_slope_mu(biomass_R1_exponential_phase, sample_timepoints_R1_exponential_phase):
    biomass_R1_exponential_phase = pd.DataFrame(dict(biomass_R1_exponential_phase = biomass_R1_exponential_phase, sample_timepoints_R1_exponential_phase = sample_timepoints_R1_exponential_phase))
    biomass_R1_exponential_phase.reset_index(inplace=True, drop=True)
    biomass_R1_exponential_phase = biomass_R1_exponential_phase[np.isfinite(biomass_R1_exponential_phase['biomass_R1_exponential_phase'])]
    # Linear regression model
    model = LinearRegression(fit_intercept=True)
    model.fit(biomass_R1_exponential_phase['sample_timepoints_R1_exponential_phase'][:, np.newaxis],biomass_R1_exponential_phase['biomass_R1_exponential_phase'])

    
    first_index = biomass_R1_exponential_phase['sample_timepoints_R1_exponential_phase'].index[0]
    last_index = biomass_R1_exponential_phase['sample_timepoints_R1_exponential_phase'].index[-1]

    xfit = np.linspace(biomass_R1_exponential_phase['sample_timepoints_R1_exponential_phase'][first_index], biomass_R1_exponential_phase['sample_timepoints_R1_exponential_phase'][last_index], 1000)
    yfit = model.predict(xfit[:, np.newaxis])

    plt.scatter(biomass_R1_exponential_phase['sample_timepoints_R1_exponential_phase'], biomass_R1_exponential_phase['biomass_R1_exponential_phase'])
    plt.plot(xfit, yfit)
    plt.show()



    print("Model slope (mu_max):    ", model.coef_[0])
    

    # Finds the first and last index to make a model in between 1000 steps
    #first_index = biomass_R1_exponential_phase[str(sample_timepoints_R_exponential_phase)].index[0]
    #last_index = biomass_R1_exponential_phase[str(sample_timepoints_R_exponential_phase)].index[-1]
    
